# Snowflake Account Usage - Workspace Version

### 이 앱은 여러분의 Snowflake 계정에 있는 account_usage 스키마를 기반으로 작동하도록 개발되었습니다.

이 노트북은 Snowflake Notebooks에서 VS Code Workspace용으로 변환된 버전입니다.
SQL 셀의 결과는 자동으로 dataframe_X 형태로 할당됩니다.

자세한 정보는 아래 Snowflake 공식 문서 페이지를 참조하십시오.  
https://docs.snowflake.com/en/sql-reference/account-usage#account-usage-views

# 패키지 설정 및 측정 날짜 지정

In [ ]:
import pandas as pd
import plotly.express as px
import datetime
import altair as alt
import plotly.graph_objects as go
import numpy as np

#alt.data_transformers.enable("vegafusion")
#alt.renderers.enable("mimetype")
    
from IPython.display import display, Markdown

from snowflake.snowpark.context import get_active_session

session = get_active_session()

def title(text):
    display(Markdown(f"## {text}"))

In [ ]:
title('모니터링 날짜 범위 지정')

DAYS = 30

today = datetime.date.today()
s = (today - datetime.timedelta(days=DAYS)).strftime('%Y-%m-%d')
e = today.strftime('%Y-%m-%d')

print(f"분석 기간: {s} ~ {e}")
print(f"\n※ 기간 변경하려면 위의 DAYS 값을 수정하세요 (7, 14, 30, 60, 120)")

# 사용량 개요

In [ ]:
%%sql -r dataframe_1
--Credits Used
select round(sum(credits_used),0) as total_credits 
  from snowflake.account_usage.metering_history 
 where start_time between '{{s}}' and '{{e}}' ;

In [ ]:
%%sql -r dataframe_2
--Total # of Jobs Executed
select count(*) as number_of_jobs 
  from snowflake.account_usage.query_history 
 where start_time between '{{s}}' and '{{e}}' ;

In [ ]:
%%sql -r dataframe_3
--Current Storage
select round(avg(storage_bytes + stage_bytes + failsafe_bytes) / power(1024, 4),2) as billable_tb 
  from snowflake.account_usage.storage_usage 
 where USAGE_DATE = current_date() - 1;

In [ ]:
title('Account 사용량 개요')

pandas_credits_used_df = dataframe_1
pandas_num_jobs_df = dataframe_2
pandas_current_storage_df = dataframe_3

credits_used_tile = pandas_credits_used_df.iloc[0].values[0]
num_jobs_tile = pandas_num_jobs_df.iloc[0].values[0]
current_storage_tile = pandas_current_storage_df.iloc[0].values[0]

print(f"📊 Credits Used: {int(credits_used_tile):,}")
print(f"📊 Total # of Jobs Executed: {int(num_jobs_tile):,}")
print(f"📊 Current Storage (TB): {current_storage_tile}")

In [ ]:
%%sql -r dataframe_4
--Credits Billed by Month 
select date_trunc('MONTH', usage_date) as Usage_Month, 
       sum(CREDITS_BILLED) as sum_credits
  from snowflake.account_usage.metering_daily_history 
 WHERE usage_date >= DATEADD('MONTH', -12, DATE_TRUNC('MONTH', CURRENT_TIMESTAMP()))
 group by Usage_Month ;

In [ ]:
title('월별 전체 크레딧 사용량 추이')

credits_billed_df = dataframe_4
fig_credits_billed=px.bar(credits_billed_df,x='USAGE_MONTH',y='SUM_CREDITS', orientation='v',title="Credits Billed by Month")
fig_credits_billed.show()

print('The above chart is static and not modified by the date range filter')

In [ ]:
%%sql -r dataframe_5
--Total # of Jobs Executed by Month 
SELECT DATE_TRUNC('MONTH', START_TIME) AS job_month, 
       COUNT(*) AS number_of_jobs                   
  FROM snowflake.account_usage.query_history       
 WHERE START_TIME >= DATEADD('MONTH', -12, DATE_TRUNC('MONTH', CURRENT_TIMESTAMP()))        
 GROUP BY 1                                            
 ORDER BY 1 ASC                                        
;

In [ ]:
title('월별 전체 Job 수행 수 변화 추이')

JobByMonth_df = dataframe_5
fig_JobByMonth=px.bar(JobByMonth_df,x='JOB_MONTH',y='NUMBER_OF_JOBS', orientation='v',title="Jobs by Month")
fig_JobByMonth.show()

print('The above chart is static and not modified by the date range filter')

# 가상 웨어하우스 (Virtual Warehouse) 분석

In [ ]:
%%sql -r dataframe_6
-- Credits Usages of Warehouse
select warehouse_name,sum(credits_used) as total_credits_used 
  from snowflake.account_usage.warehouse_metering_history 
 where start_time between '{{s}}' and '{{e}}' 
 group by 1 
 order by 2 desc 
 limit 10 ;

In [ ]:
title('지정된 기간 웨어하우스별 사용된 크레딧')

pandas_credits_used_df = dataframe_6
pandas_credits_used_df_sorted = pandas_credits_used_df.sort_values(by='TOTAL_CREDITS_USED', ascending=True)

fig_credits_used=px.bar(pandas_credits_used_df_sorted,x='TOTAL_CREDITS_USED',y='WAREHOUSE_NAME',orientation='h',title="Credits Used by Warehouse")
fig_credits_used.update_traces(marker_color='green')
fig_credits_used.show()

In [ ]:
%%sql -r dataframe_7
--Jobs by Warehouse Data Setup
select warehouse_name,count(*) as number_of_jobs 
  from snowflake.account_usage.query_history 
 where start_time between '{{s}}' and '{{e}}' 
   and warehouse_name is not null
 group by 1 
 order by 2 desc 
 limit 10 ;

In [ ]:
title('지정된 기간 웨어하우스별 수행된 작업의 수')

pandas_jobs_by_warehouse_df = dataframe_7
pandas_jobs_by_warehouse_df_sorted = pandas_jobs_by_warehouse_df.sort_values(by='NUMBER_OF_JOBS', ascending=True)

fig_jobs_by_warehouse=px.bar(pandas_jobs_by_warehouse_df_sorted,x='NUMBER_OF_JOBS',y='WAREHOUSE_NAME',orientation='h',title="# of Jobs by Warehouse")
fig_jobs_by_warehouse.update_traces(marker_color='purple')
fig_jobs_by_warehouse.show()

In [ ]:
%%sql -r dataframe_8
--Average Execution by Query Type
select query_type, warehouse_size, avg(execution_time) / 1000 as average_execution_time 
  from snowflake.account_usage.query_history 
 where start_time between '{{s}}' and '{{e}}' 
 group by 1, 2 
 order by 3 desc;

In [ ]:
title('지정된 기간 쿼리 유형별, 웨어하우스 크기별 평균 실행 시간 (초)')

pandas_execution_by_qtype_df = dataframe_8
fig_execution_by_qtype=px.bar(pandas_execution_by_qtype_df,x='AVERAGE_EXECUTION_TIME',y='QUERY_TYPE',color='WAREHOUSE_SIZE',orientation='h',title="Average Execution Time by Query Type and Warehouse Size")
fig_execution_by_qtype.show()

In [ ]:
%%sql -r dataframe_9
--Credits Used Overtime
select start_time::date as usage_date, warehouse_name, sum(credits_used) as total_credits_used 
  from snowflake.account_usage.warehouse_metering_history 
 where start_time between '{{s}}' and '{{e}}' 
 group by 1,2 
having total_credits_used > 0.001
 order by 2,1 ;

In [ ]:
title('지정된 기간 웨어하우스별 크레딧 사용양 추세')

pandas_credits_used_overtime_df = dataframe_9
fig_credits_used_overtime_df=px.bar(pandas_credits_used_overtime_df,x='USAGE_DATE',y='TOTAL_CREDITS_USED',color='WAREHOUSE_NAME',orientation='v',title="Credits Used Overtime")
fig_credits_used_overtime_df.show()

In [ ]:
%%sql -r dataframe_10
--Warehouse Variance Overtime
SELECT WAREHOUSE_NAME, 
       DATE(START_TIME) AS DATE, 
       SUM(CREDITS_USED) AS CREDITS_USED, 
       AVG(SUM(CREDITS_USED)) OVER (PARTITION BY WAREHOUSE_NAME ORDER BY DATE ROWS 7 PRECEDING) 
         AS CREDITS_USED_7_DAY_AVG, 
       (TO_NUMERIC(SUM(CREDITS_USED)/CREDITS_USED_7_DAY_AVG*100,10,2)-100)::STRING || '%' 
         AS VARIANCE_TO_7_DAY_AVERAGE 
  FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY 
 where start_time between '{{s}}' and '{{e}}' 
 GROUP BY DATE, WAREHOUSE_NAME 
 ORDER BY DATE DESC

In [ ]:
title('지정된 기간내의 해당 날짜별 이전 7일간의 평균 웨어하우스 사용량 대비 증감양')

pandas_warehouse_variance_df = dataframe_10
fig_warehouse_variance_df=px.bar(pandas_warehouse_variance_df,x="DATE",y="VARIANCE_TO_7_DAY_AVERAGE",color ='WAREHOUSE_NAME',orientation='v',title="Warehouse Variance Greater than 7 day Average")
fig_warehouse_variance_df.show()

In [ ]:
%%sql -r dataframe_11
with base_table as (
select warehouse_name, 
       total_elapsed_time,
       case
            when total_elapsed_time < 1 then 'a'
            when total_elapsed_time < 10 then 'b'
            when total_elapsed_time < 60 then 'c'
            when total_elapsed_time < 300 then 'd'
            when total_elapsed_time < 600 then 'e'
            when total_elapsed_time > 600 then 'f'
       end as exe_time
  from SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
 where warehouse_name is not null
   and start_time between '{{s}}' and '{{e}}'
   and warehouse_name not ilike 'compute_service%'   
)
select warehouse_name, 
       sum(iff(exe_time= 'a', 1, 0)) u00_01s,
       sum(iff(exe_time= 'b', 1, 0)) u01_10s,
       sum(iff(exe_time= 'c', 1, 0)) u10_60s,
       sum(iff(exe_time= 'd', 1, 0)) u01_05m,
       sum(iff(exe_time= 'e', 1, 0)) u05_10m,
       sum(iff(exe_time= 'f', 1, 0)) u10m_
  from base_table
 group by warehouse_name;

In [ ]:
title('지정된 기간 동안 웨어하우스별 워크로드 수행시간 분포')

ExeTimeInWH_df = dataframe_11

plot_columns = ['U00_01S', 'U01_10S', 'U10_60S', 'U01_05M', 'U05_10M', 'U10M_']

labels_map = {
    'WAREHOUSE_NAME': 'Warehouse Name',
    'variable': 'Execution Time Bucket',
    'value': 'Number of Queries',
    'U00_01S': '< 1s',
    'U01_10S': '1s - 10s',
    'U10_60S': '10s - 60s',
    'U01_05M': '1m - 5m',
    'U05_10M': '5m - 10m',
    'U10M_': '> 10m'
}

df_long = pd.melt(ExeTimeInWH_df,
                  id_vars='WAREHOUSE_NAME',
                  value_vars=plot_columns,
                  var_name='variable',
                  value_name='value')

fig = px.bar(
    df_long,
    x='WAREHOUSE_NAME',
    y='value',
    color='variable',
    title='Query Execution Time Distribution by Warehouse',
    labels=labels_map,
    height=500,
    category_orders={'variable': plot_columns}
)

fig.update_yaxes(tickformat=',')
fig.show()

In [ ]:
%%sql -r dataframe_12
select warehouse_name,
       percentile_cont(0.25) within group(order by query_load_percent) as p25,
       median(query_load_percent) as md,
       percentile_cont(0.75) within group(order by query_load_percent) as p75
  from snowflake.account_usage.query_history
 where warehouse_name is not null
   and query_load_percent is not null
   and start_time between '{{s}}' and '{{e}}'
   and warehouse_name not ilike 'compute_service%'
 group by 1
 order by 3 desc ;

In [ ]:
title('지정된 기간 웨어하우스별 워크로드 사용량 분포 (25%, 50%, 75%)')

WHWorkloadPCT_df = dataframe_12

WHWorkloadPCT_df['error_y_upper'] = WHWorkloadPCT_df['P75'] - WHWorkloadPCT_df['MD']
WHWorkloadPCT_df['error_y_lower'] = WHWorkloadPCT_df['MD']  - WHWorkloadPCT_df['P25']

fig = px.scatter(
    WHWorkloadPCT_df,
    x='WAREHOUSE_NAME',
    y='MD',
    error_y='error_y_upper',
    error_y_minus='error_y_lower',
    title='Warehouse Query Load Distribution (Median and P25-P75 Range)',
    labels={
        'WAREHOUSE_NAME': 'Warehouse',
        'MD': 'Median Query Load (%)',
        'P25': 'P25 Load (%)',
        'P75': 'P75 Load (%)'
    },
    hover_data=['P25', 'MD', 'P75']
)

fig.update_layout(
    yaxis_title='Query Load Percent (%)',
    xaxis_title='Warehouse Name',
    yaxis_range=[0, max(100, WHWorkloadPCT_df['P75'].max() * 1.1)],
    yaxis_ticksuffix='%'
)

fig.update_traces(
    error_y_thickness=1,
    error_y_width=5
)

fig.show()

In [ ]:
%%sql -r dataframe_13
SELECT to_date(start_time) as date,
       warehouse_name,
       sum(avg_running) as sum_running,
       sum(avg_queued_load) as sum_queued
  FROM snowflake.account_usage.warehouse_load_history
 WHERE to_date(start_time) >= dateadd(month, -1, current_timestamp())
 GROUP BY 1,2
 ORDER BY 1,2 ;

In [ ]:
title('일별 웨어하우스의 워크로드 추이 (지난 1개월)')

RunQueueWH_df = dataframe_13

fig_running = px.line(
    RunQueueWH_df,
    x='DATE',
    y='SUM_RUNNING',
    color='WAREHOUSE_NAME',
    title='Daily Sum of Average Running Queries by Warehouse',
    labels={
        'DATE': 'Date',
        'SUM_RUNNING': 'Sum of Avg Running Queries (Daily)',
        'WAREHOUSE_NAME': 'Warehouse'
    },
    markers=True,
    hover_data={'WAREHOUSE_NAME': True, 'SUM_RUNNING': ':.1f'}
)
fig_running.update_layout(
    xaxis_title='Date',
    yaxis_title='Daily Sum of Avg Running Queries',
    hovermode='x unified'
)
fig_running.show()

fig_queued = px.line(
    RunQueueWH_df,
    x='DATE',
    y='SUM_QUEUED',
    color='WAREHOUSE_NAME',
    title='Daily Sum of Average Queued Queries by Warehouse',
    labels={
        'DATE': 'Date',
        'SUM_QUEUED': 'Sum of Avg Queued Queries (Daily)',
        'WAREHOUSE_NAME': 'Warehouse'
    },
    markers=True,
    hover_data={'WAREHOUSE_NAME': True, 'SUM_QUEUED': ':.1f'}
)
fig_queued.update_layout(
    xaxis_title='Date',
    yaxis_title='Daily Sum of Avg Queued Queries',
    hovermode='x unified'
)
min_queued = RunQueueWH_df['SUM_QUEUED'].min()
max_queued = RunQueueWH_df['SUM_QUEUED'].max()
fig_queued.update_yaxes(range=[min_queued, max(1, max_queued * 1.1)])
fig_queued.show()

# 쿼리 분석

In [ ]:
%%sql -r dataframe_14
--Top 25 Longest Success Queries
select query_id,query_text,(execution_time / 1000) as exec_time 
  from snowflake.account_usage.query_history 
 where execution_status = 'SUCCESS' 
   and start_time between '{{s}}' and '{{e}}' 
   and query_text != ''
   and exec_time > 10
 order by execution_time desc 
 limit 25;

In [ ]:
title('지정된 기간동안 가장 오래 수행된 쿼리 리스트')

pandas_longest_queries_df = dataframe_14

max_len = 100
col_orig = 'QUERY_TEXT'
col_short = 'QUERY_TEXT_SHORT'

pandas_longest_queries_df[col_short] = pandas_longest_queries_df[col_orig].str.slice(0, max_len)
pandas_longest_queries_df.loc[pandas_longest_queries_df[col_orig].str.len() > max_len, col_short] += '...'

fig_longest_queries = px.bar(
    pandas_longest_queries_df,
    x='EXEC_TIME',
    y=col_short,
    orientation='h',
    title="Longest Successful Queries (Top 25)",
    hover_data={
        'EXEC_TIME': ':.2f s',
        col_orig: True,
        col_short: False
    },
    labels={col_short: "Query Text (Truncated)"}
)

fig_longest_queries.show()

In [ ]:
%%sql -r dataframe_15
--Top 25 Failed Queries 
select query_text,count(*) as number_of_execution  
  from snowflake.account_usage.query_history 
 where execution_status = 'FAIL' 
   and start_time between '{{s}}' and '{{e}}' 
   and query_text != ''
 group by 1
having number_of_execution > 1
 order by 2 desc 
 limit 25;

In [ ]:
title('지정된 기간동안 가장 많이 실패한 쿼리 문 리스트')

f_pandas_longest_queries_df = dataframe_15

max_len = 100
col_orig = 'QUERY_TEXT'
col_short = 'QUERY_TEXT_SHORT'

f_pandas_longest_queries_df[col_short] = f_pandas_longest_queries_df[col_orig].str.slice(0, max_len)
f_pandas_longest_queries_df.loc[f_pandas_longest_queries_df[col_orig].str.len() > max_len, col_short] += '...'

fig_f_longest_queries = px.bar(
    f_pandas_longest_queries_df,
    x='NUMBER_OF_EXECUTION',
    y=col_short,
    orientation='h',
    title="Most Failed Queries (Top 25)",
    hover_data={
        'NUMBER_OF_EXECUTION': ':3d s', 
        col_orig: True,
        col_short: False
    },
    labels={col_short: "Query Text (Truncated)"}
)
fig_f_longest_queries.update_traces(marker_color='red')

fig_f_longest_queries.show()

In [ ]:
%%sql -r dataframe_16
--Total Execution Time by Repeated Queries
select query_text, 
       (sum(execution_time) / 1000) as exec_time,
       count(*) as number_of_query
  from snowflake.account_usage.query_history 
 where execution_status = 'SUCCESS' 
   and start_time between '{{s}}' and '{{e}}' 
   and query_text != ''
 group by query_text 
 order by exec_time desc 
 limit 10 ;

In [ ]:
title('지정된 기간내의 반복되는 쿼리 수행')

Total_Execution_Time_df = dataframe_16

max_len = 100
col_orig = 'QUERY_TEXT'
col_short = 'QUERY_TEXT_SHORT'

Total_Execution_Time_df[col_short] = Total_Execution_Time_df[col_orig].str.slice(0, max_len)
Total_Execution_Time_df.loc[Total_Execution_Time_df[col_orig].str.len() > max_len, col_short] += '...'

fig_Total_Execution_Time = px.bar(
    Total_Execution_Time_df,
    x='EXEC_TIME',
    y=col_short,
    orientation='h',
    title="Total Execution Time by Repeated Queries",
    hover_data={
        'EXEC_TIME': ':.3f s',
        col_orig: True,
        col_short: False,
        'NUMBER_OF_QUERY': ':,d'
    },
    labels={
        col_short: "Query Text (Truncated)",
        'EXEC_TIME': "Total Execution Time (s)",
        'NUMBER_OF_QUERY': "Number of Executions"
    }
)
fig_Total_Execution_Time.update_traces(marker_color='LightSkyBlue')

fig_Total_Execution_Time.show()

In [ ]:
%%sql -r dataframe_17
--QUERIES THAT SPILL TO DISK
select top 25
       query_text
      ,avg(total_elapsed_time) as avg_total_elapsed_time
      ,sum(bytes_spilled_to_local_storage) / 1024 / 1024 / 1024 as sum_gb_spilled_to_disk
      ,sum(bytes_spilled_to_remote_storage) / 1024 / 1024 / 1024 as sum_gb_spilled_to_blob
      ,sum(bytes_spilled_to_local_storage+bytes_spilled_to_remote_storage) as total_bytes
  from snowflake.account_usage.query_history
 where start_time between '{{s}}' and '{{e}}' 
   and query_text != ''
 group by query_text
having total_bytes > 0
 order by total_bytes desc;

In [ ]:
title('지정된 기간 동안 Disk로 Spill된 쿼리 리스트')

SpillToDisk_df = dataframe_17

max_len = 100
col_orig = 'QUERY_TEXT'
col_short = 'QUERY_TEXT_SHORT'

SpillToDisk_df[col_short] = SpillToDisk_df[col_orig].str.slice(0, max_len)
SpillToDisk_df.loc[SpillToDisk_df[col_orig].str.len() > max_len, col_short] += '...'

fig_SpillToDisk = px.bar(
    SpillToDisk_df,
    x='TOTAL_BYTES',
    y=col_short,
    orientation='h',
    title="Spill to Disk Queries",
    hover_data={
        'AVG_TOTAL_ELAPSED_TIME': ':.2f',  
        'TOTAL_BYTES': ':.d',  
        col_orig: True,
        col_short: False
    },
    labels={
        col_short: "Query Text (Truncated)",
        'TOTAL_BYTES': "Spill to Disk",
        'AVG_TOTAL_ELAPSED_TIME': "Average Elapsed Time"
    }
)
fig_SpillToDisk.update_traces(marker_color='Yellow')

fig_SpillToDisk.show()

In [ ]:
%%sql -r dataframe_18
--Top 10 Average Query Execution Time (By User)
select user_name, (avg(execution_time)) / 1000 as average_execution_time 
  from snowflake.account_usage.query_history 
 where execution_status = 'SUCCESS'
   and start_time between '{{s}}' and '{{e}}'
 group by 1 
 order by 2 desc limit 10 ;

In [ ]:
title('지정된 기간동안 사용자별 평균 쿼리 실행 시간 (초)')

query_execution_df = dataframe_18
fig_cquery_execution=px.bar(query_execution_df,x='USER_NAME',y='AVERAGE_EXECUTION_TIME', orientation='v',title="Average Execution Time per User")
fig_cquery_execution.update_traces(marker_color='MediumPurple')
fig_cquery_execution.show()

In [ ]:
%%sql -r dataframe_19
SELECT query_history.query_id,
       query_history.query_text,
       query_history.start_time,
       query_history.end_time,
       query_history.user_name,
       query_history.database_name,
       query_history.schema_name,
       query_history.warehouse_name,
       query_history.warehouse_size,
       metering_history.credits_used,
       execution_time/1000 as execution_time_s
  FROM snowflake.account_usage.query_history
  JOIN snowflake.account_usage.metering_history 
    ON query_history.start_time >= metering_history.start_time
   AND query_history.end_time <= metering_history.end_time
 WHERE query_history.start_time >= DATEADD(DAY, -7, CURRENT_TIMESTAMP())
 ORDER BY query_history.query_id;

In [ ]:
title('한 주간 시간대별 쿼리 사용량 - Heatmap')

df = dataframe_19

days = 7 
var = 'WAREHOUSE_NAME'

df['START_TIME'] = pd.to_datetime(df['START_TIME']) 
latest_date = df['START_TIME'].max() 
cutoff_date = latest_date - pd.Timedelta(days=days) 
filtered_df = df[df['START_TIME'] > cutoff_date].copy()

filtered_df['HOUR_OF_DAY'] = filtered_df['START_TIME'].dt.hour 
filtered_df['HOUR_DISPLAY'] = filtered_df['HOUR_OF_DAY'].apply(lambda x: f"{x:02d}:00")

agg_df = (filtered_df.groupby(['HOUR_DISPLAY', var]) .agg(COUNT=('QUERY_ID', 'count')) .reset_index() )

pivot_df = agg_df.pivot(index=var, columns='HOUR_DISPLAY', values='COUNT').fillna(0)

print(f"Analyzing {var} data for the last {days} days!")

fig = px.imshow( pivot_df, labels=dict(x="Hour", y="Warehouse", color="Query Count"), title=f'Query Activity Heatmap by Hour and {var}', aspect="auto", color_continuous_scale="Blues" ) 
fig.update_layout(height=max(400, len(pivot_df) * 30)) 
fig.show()

In [ ]:
%%sql -r dataframe_20
--GS Utilization by Query Type (Top 10)
select query_type, 
       sum(credits_used_cloud_services) cs_credits, 
       count(1) num_queries 
  from snowflake.account_usage.query_history 
 where true 
   and start_time between '{{s}}' and '{{e}}'
 group by 1 
 order by 2 desc 
 limit 10 ;

In [ ]:
title('지정된 기간동안 쿼리 유형별 클라우드서비스 총 사용 크레딧량')

gs_utilization_df = dataframe_20

fig_gs_utilization = px.bar(
    gs_utilization_df,
    x='QUERY_TYPE',
    y='CS_CREDITS',
    orientation='v',
    title="GS Utilization by Query Type (Top 10)",
    hover_data={
        'QUERY_TYPE': False,
        'CS_CREDITS': ':.2f credits',
        'NUM_QUERIES': ':,d'
    },
    labels={
        'CS_CREDITS': 'Cloud Service Credits',
        'NUM_QUERIES': 'Number of Queries'
    }
)
fig_gs_utilization.update_traces(marker_color='green')
fig_gs_utilization.show()

In [ ]:
%%sql -r dataframe_21
--Top 10 Cloud Services by Warehouse   
select warehouse_name, 
       sum(credits_used_cloud_services) CREDITS_USED_CLOUD_SERVICES 
  from snowflake.account_usage.warehouse_metering_history 
 where true 
   and start_time between '{{s}}' and '{{e}}'
 group by 1 
 order by 2 desc limit 10;

In [ ]:
title('지정된 기간동안 웨어하우스별 클라우드서비스의 총 사용 크레딧양')

compute_gs_by_warehouse_df = dataframe_21
fig_compute_gs_by_warehouse=px.bar(compute_gs_by_warehouse_df,x='WAREHOUSE_NAME',y='CREDITS_USED_CLOUD_SERVICES', orientation='v',title="Compute and Cloud Services by Warehouse", barmode="group")
fig_compute_gs_by_warehouse.update_traces(marker_color='purple')
fig_compute_gs_by_warehouse.show()

# 스토리지

In [ ]:
%%sql -r dataframe_22
--Data Storage used Overtime 
select date_trunc(month, usage_date) as usage_month, 
       avg(storage_bytes + stage_bytes + failsafe_bytes) / power(1024, 4) as billable_tb, 
       avg(storage_bytes) / power(1024, 4) as Storage_TB, 
       avg(stage_bytes) / power(1024, 4) as Stage_TB, 
       avg(failsafe_bytes) / power(1024, 4) as Failsafe_TB 
  from snowflake.account_usage.storage_usage 
 group by 1 
 order by 1 ;

In [ ]:
title('월별 스토리지 사용량 추이')

storage_overtime_df = dataframe_22

storage_df_melted = pd.melt(
    storage_overtime_df,
    id_vars=['USAGE_MONTH'],
    value_vars=['STORAGE_TB', 'STAGE_TB', 'FAILSAFE_TB'],
    var_name='STORAGE_TYPE',
    value_name='SIZE_TB'
)

storage_type_order = ['STORAGE_TB', 'STAGE_TB', 'FAILSAFE_TB']
storage_df_melted['STORAGE_TYPE'] = pd.Categorical(
    storage_df_melted['STORAGE_TYPE'], categories=storage_type_order, ordered=True
)
storage_df_melted = storage_df_melted.sort_values(by=['USAGE_MONTH', 'STORAGE_TYPE'])

fig_stacked_storage = px.bar(
    storage_df_melted,
    x='USAGE_MONTH',
    y='SIZE_TB',
    color='STORAGE_TYPE',
    title="Data Storage used Overtime",
    orientation='v',
    hover_data={
        'USAGE_MONTH': True,
        'STORAGE_TYPE': True,
        'SIZE_TB': ':.2f TB'
    },
    labels={
        'USAGE_MONTH': 'Month',
        'SIZE_TB': 'Storage Size (TB)',
        'STORAGE_TYPE': 'Storage Type'
    }
)

fig_stacked_storage.show()
print('The above chart is static and not modified by the date range filter')

In [ ]:
%%sql -r dataframe_23
--Rows Loaded Overtime (COPY INTO) 
select to_timestamp(date_trunc(day,last_load_time)) as usage_date, 
       sum(row_count) as total_rows 
  from snowflake.account_usage.load_history 
 where usage_date between '{{s}}' and '{{e}}' 
 group by 1 
 order by usage_date desc ;

In [ ]:
title('지정된 기간동안 일별 데이터 적재건수 추이')

rows_loaded_df = dataframe_23

fig_rows_loaded=px.bar(rows_loaded_df,x='USAGE_DATE',y='TOTAL_ROWS', orientation='v',title="Rows Loaded Overtime (Copy Into)")
fig_rows_loaded.show()

# 사용자

In [ ]:
%%sql -r dataframe_24
--Logins by User 
select user_name, 
       sum(iff(is_success = 'NO', 1, 0)) as Failed, 
       count(*) as Success, 
       sum(iff(is_success = 'NO', 1, 0)) / nullif(count(*), 0) as login_failure_rate 
  from snowflake.account_usage.login_history 
 where true
   and event_timestamp between '{{s}}' and '{{e}}'
 group by 1 
 order by 4 desc;

In [ ]:
title('지정된 기간동안 사용자별 로그인 성공/실패 횟수')

logins_df_wide = dataframe_24

logins_df_melted = pd.melt(
    logins_df_wide,
    id_vars=['USER_NAME'],
    value_vars=['SUCCESS', 'FAILED'],
    var_name='LOGIN_STATUS',
    value_name='COUNT'
)

status_order = ['SUCCESS', 'FAILED']
logins_df_melted['LOGIN_STATUS'] = pd.Categorical(
    logins_df_melted['LOGIN_STATUS'], categories=status_order, ordered=True
)
logins_df_melted = logins_df_melted.sort_values(by=['USER_NAME', 'LOGIN_STATUS'])

fig_logins_stacked = px.bar(
    logins_df_melted,
    x='USER_NAME',
    y='COUNT',
    color='LOGIN_STATUS',
    title="Sucessful & Failed login by Users",
    orientation='v',
    hover_data={
        'USER_NAME': True,
        'LOGIN_STATUS': True,
        'COUNT': ':,d'
    },
    labels={
        'USER_NAME': 'User Name',
        'COUNT': 'Login Count',
        'LOGIN_STATUS': 'Login Status'
    },
    color_discrete_map={
        'SUCCESS': 'lightgreen',
        'FAILED': 'red'
    }
)

fig_logins_stacked.show()

In [ ]:
%%sql -r dataframe_25
--Logins by Client 
select reported_client_type as Client, 
       user_name, 
       sum(iff(is_success = 'NO', 1, 0)) as Failed, 
       count(*) as Success, 
       sum(iff(is_success = 'NO', 1, 0)) / nullif(count(*), 0) as login_failure_rate 
  from snowflake.account_usage.login_history 
 where true
   and event_timestamp between '{{s}}' and '{{e}}'
 group by 1, 2 
 order by 5 desc ;

In [ ]:
title('지정된 기간동안 클라이언트 유형별, 사용자별 로그인 성공/실패 횟수')

logins_client_df_wide = dataframe_25

logins_client_df_melted = pd.melt(
    logins_client_df_wide,
    id_vars=['CLIENT', 'USER_NAME'],
    value_vars=['SUCCESS', 'FAILED'],
    var_name='LOGIN_STATUS',
    value_name='COUNT'
)

status_order = ['SUCCESS', 'FAILED']
logins_client_df_melted['LOGIN_STATUS'] = pd.Categorical(
    logins_client_df_melted['LOGIN_STATUS'], categories=status_order, ordered=True
)
logins_client_df_melted = logins_client_df_melted.sort_values(by=['CLIENT', 'USER_NAME', 'LOGIN_STATUS'])

fig_logins_client_grouped = px.bar(
    logins_client_df_melted,
    x='USER_NAME',
    y='COUNT',
    color='LOGIN_STATUS',
    facet_col='CLIENT',
    facet_col_wrap=3,
    title="Logins by Client",
    hover_data={
        'CLIENT': True,
        'USER_NAME': True,
        'LOGIN_STATUS': True,
        'COUNT': ':,d'
    },
    labels={
        'USER_NAME': 'Username',
        'COUNT': 'Number of logins',
        'LOGIN_STATUS': 'Login status',
        'CLIENT': 'Client type'
    },
    color_discrete_map={
        'SUCCESS': 'mediumseagreen',
        'FAILED': 'tomato'
    },
    height=500
)

fig_logins_client_grouped.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_logins_client_grouped.update_xaxes(tickangle=45)

fig_logins_client_grouped.show()

In [ ]:
%%sql -r dataframe_26
--Never Logged In Users
SHOW USERS;

In [ ]:
SELECT * FROM {{dataframe_26}} 
WHERE last_success_login IS NULL
AND DATEDIFF('Day',created_on,CURRENT_DATE) > 30;

In [ ]:
title('생성 후 30일 이내 로그인한 이력이 없는 사용자')

df_users = dataframe_27

if not df_users.empty:
    df_users['CREATED_ON'] = pd.to_datetime(df_users['CREATED_ON'])

    print("사용자 생성 타임라인")

    fig_timeline = px.scatter(
        df_users, 
        x='CREATED_ON',     # 대문자로 변경
        y='NAME',           # 대문자로 변경
        title='Inactive User Creation Timeline',
        labels={            
            'CREATED_ON': 'Creation Date/Time',  # 대문자로 변경
            'NAME': 'User Name'                   # 대문자로 변경
        },
        hover_name='NAME',  # 대문자로 변경
        hover_data={       
            'CREATED_ON': '|%Y-%m-%d %H:%M:%S',  # 대문자로 변경
            'NAME': False                         # 대문자로 변경
        }
    )

    fig_timeline.update_layout(
        xaxis_title='Creation Date/Time',
        yaxis_title='User Name',
        height=max(400, len(df_users['NAME'].unique()) * 20)  # 대문자로 변경
    )
    fig_timeline.update_yaxes(categoryorder='category ascending')
    fig_timeline.update_traces(marker=dict(size=8, symbol='circle'))

    fig_timeline.show()
else:
    print("조건에 맞는 사용자가 없습니다.")

In [ ]:
%%sql -r dataframe_28
--Stale Users
SHOW USERS;

In [ ]:
SELECT * FROM {{dataframe_28}}
WHERE DATEDIFF('Day',last_success_login,CURRENT_DATE) > 30;

In [ ]:
title('지난 30일 이내 새로 로그인하지 않은 사용자')

df_users = dataframe_29

if not df_users.empty:
    df_users['LAST_SUCCESS_LOGIN'] = pd.to_datetime(df_users['LAST_SUCCESS_LOGIN'])

    print("사용자 마지막 로그인 타임라인")

    fig_timeline = px.scatter(
        df_users, 
        x='LAST_SUCCESS_LOGIN',     
        y='NAME',           
        title='Infrequent User',
        labels={            
            'LAST_SUCCESS_LOGIN': 'Last Success Login Date/Time',
            'NAME': 'User Name'
        },
        hover_name='NAME',  
        hover_data={       
            'LAST_SUCCESS_LOGIN': '|%Y-%m-%d %H:%M:%S', 
            'NAME': False 
        }
    )

    fig_timeline.update_layout(
        xaxis_title='Last Success Login Date/Time',
        yaxis_title='User Name',
        height=max(400, len(df_users['NAME'].unique()) * 20)
    )
    fig_timeline.update_yaxes(categoryorder='category ascending')
    fig_timeline.update_traces(marker=dict(size=8, symbol='circle'))

    fig_timeline.show()
else:
    print("조건에 맞는 사용자가 없습니다.")

# 테이블 분석

In [ ]:
%%sql -r dataframe_30
-- Identify high churn tables or short lived tables
SELECT
        t.table_catalog||'.'||t.table_schema||'.'||t.table_name as fq_table_name
       ,t.table_catalog as database
       ,t.table_schema as schema
       ,t.table_name as table_name
       ,t.active_bytes/power(1024,3) as active_size_gb
       ,t.time_travel_bytes/power(1024,3) as time_travel_gb
       ,t.failsafe_bytes/power(1024,3) as failsafe_gb
       ,t.retained_for_clone_bytes/power(1024,3) as clone_retain_gb
       ,active_size_gb+time_travel_gb+failsafe_gb+clone_retain_gb as total_size_gb
       ,(t.time_travel_bytes + t.failsafe_bytes + t.retained_for_clone_bytes)/power(1024,3) as non_active_size_gb
       ,div0(non_active_size_gb,active_size_gb)*100 as churn_pct
       ,t.deleted
       ,timediff('hour',t.table_created,t.table_dropped) as table_life_duration_hours
       ,t1.is_transient
       ,t1.table_type
       ,t1.retention_time
       ,t1.auto_clustering_on
       ,t1.clustering_key
       ,t1.last_altered
       ,t1.last_ddl
   FROM snowflake.account_usage.table_storage_metrics t
        JOIN snowflake.account_usage.tables t1
          ON t.id=t1.table_id
  WHERE 1=1
        AND t1.table_catalog not in ('SNOWFLAKE')
        AND 
            (
             churn_pct>=40
             OR
             table_life_duration_hours<=24
            )
  ORDER BY total_size_gb desc ;

In [ ]:
title('짧은 생명 주기의 테이블 리스트')

HighChurn_ShortLived_df = dataframe_30

print("데이터베이스별 테이블 수")
fig_db = px.bar(HighChurn_ShortLived_df, x='DATABASE', color='DATABASE',
             title='데이터베이스별 테이블 수',
             labels={'DATABASE': '데이터베이스', 'count': '테이블 수'})
fig_db.update_layout(xaxis_tickangle=-45)
fig_db.show()

print("스키마별 테이블 수")
fig_schema = px.bar(HighChurn_ShortLived_df, x='SCHEMA', color='SCHEMA',
                 title='스키마별 테이블 수',
                 labels={'SCHEMA': '스키마', 'count': '테이블 수'})
fig_schema.update_layout(xaxis_tickangle=-45)
fig_schema.show()

print("테이블별 총 크기 (GB)")
df_sorted = HighChurn_ShortLived_df.sort_values(by='TOTAL_SIZE_GB', ascending=False)
fig_size = px.bar(df_sorted, y='DATABASE', x='TOTAL_SIZE_GB', color='TABLE_NAME',
                 title='테이블별 총 크기 (GB)',
                 labels={'DATABASE': '데이터베이스', 'TOTAL_SIZE_GB': '총 크기 (GB)', 'TABLE_NAME': '테이블 이름'},
                 orientation='h')
fig_size.update_layout(yaxis={'categoryorder':'array', 'categoryarray': df_sorted['DATABASE'].unique()})
fig_size.show()

print("테이블 정보")
fig_table = go.Figure(data=[go.Table(
    header=dict(values=['DATABASE', 'SCHEMA', 'TABLE_NAME', 'DELETED','IS_TRANSIENT','LAST_ALTERED','LAST_DDL'],
                align='left'),
    cells=dict(values=[HighChurn_ShortLived_df['DATABASE'], HighChurn_ShortLived_df['SCHEMA'], HighChurn_ShortLived_df['TABLE_NAME'], HighChurn_ShortLived_df['DELETED'],HighChurn_ShortLived_df['IS_TRANSIENT'],HighChurn_ShortLived_df['LAST_ALTERED'], HighChurn_ShortLived_df['LAST_DDL']],
               align='left'))
])
fig_table.update_layout(title='테이블 이름 및 총 크기')
fig_table.show()

In [ ]:
%%sql -r dataframe_31
-- Unused tables
SELECT TABLE_CATALOG || '.' || TABLE_SCHEMA || '.' || TABLE_NAME AS TABLE_PATH
      ,TABLE_CATALOG AS DATABASE
      ,TABLE_SCHEMA AS SCHEMA
      ,TABLE_NAME
      ,BYTES
      ,TO_NUMBER(BYTES / POWER(1024,3),10,2) AS GB
      ,LAST_ALTERED AS LAST_USE
      ,DATEDIFF('Day',LAST_USE,CURRENT_DATE) AS DAYS_SINCE_LAST_USE
  FROM FROSTBYTE_TASTY_BYTES.INFORMATION_SCHEMA.TABLES
 WHERE DAYS_SINCE_LAST_USE > 7
 ORDER BY DATABASE, SCHEMA, TABLE_NAME, DAYS_SINCE_LAST_USE DESC;

In [ ]:
title('7일 이상 사용되지 않은 테이블 리스트')

UnusedTable_df = dataframe_31

df_sorted = UnusedTable_df.sort_values(by='GB', ascending=False)
fig_size = px.bar(df_sorted, y='DATABASE', x='GB', color='TABLE_PATH',
                 title='테이블별 총 크기 (GB)',
                 labels={'DATABASE': '데이터베이스', 'GB': '총 크기 (GB)', 'TABLE_PATH': '테이블 이름'},
                 orientation='h')
fig_size.update_layout(yaxis={'categoryorder':'array', 'categoryarray': df_sorted['DATABASE'].unique()})
fig_size.show()

fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(UnusedTable_df[['DATABASE', 'SCHEMA', 'TABLE_NAME','GB']].columns),
                align='left'),
    cells=dict(values=[UnusedTable_df['DATABASE'], UnusedTable_df['SCHEMA'], UnusedTable_df['TABLE_NAME'], UnusedTable_df['GB']],
               align='left'))
])
fig_table.update_layout(title='테이블 이름 및 총 크기')
fig_table.show()

In [ ]:
%%sql -r dataframe_32
-- Tables used in any query in the last 7 days
WITH access_history as
(   
SELECT  
       distinct split(base.value:objectName, '.')[0]::string as DATABASE
       ,split(base.value:objectName, '.')[1]::string as SCHEMA
       ,split(base.value:objectName, '.')[2]::string as TABLE_NAME
  FROM snowflake.account_usage.access_history 
       ,lateral flatten (base_objects_accessed) base
 where query_start_time between current_date()-7 and current_date()
)
SELECT tbl.table_catalog||'.'||tbl.table_schema||'.'||tbl.table_name as FQ_table_name,
       ah.database,
       ah.schema,
       ah.table_name,
       TO_NUMBER(tbl.bytes / POWER(1024,3),10,2) AS GB
  FROM snowflake.account_usage.tables tbl
  LEFT JOIN access_history ah
    ON tbl.table_name=ah.table_name
   AND tbl.table_schema=ah.schema
   AND tbl.table_catalog=ah.database
 WHERE ah.table_name is not NULL
   AND tbl.deleted is null 
;

In [ ]:
title('지난 7일간 쿼리에서 사용된 테이블 리스트')

TablesUsedInQuery_df = dataframe_32

df_sorted = TablesUsedInQuery_df.sort_values(by='GB', ascending=False)
fig_size = px.bar(df_sorted, y='DATABASE', x='GB', color='TABLE_NAME',
                 title='테이블별 총 크기 (GB)',
                 labels={'DATABASE': '데이터베이스', 'GB': '총 크기 (GB)', 'TABLE_NAME': '테이블 이름'},
                 orientation='h')
fig_size.update_layout(yaxis={'categoryorder':'array', 'categoryarray': df_sorted['DATABASE'].unique()})
fig_size.show()

fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(df_sorted[['DATABASE', 'SCHEMA', 'TABLE_NAME','GB']].columns),
                align='left'),
    cells=dict(values=[df_sorted['DATABASE'], df_sorted['SCHEMA'], df_sorted['TABLE_NAME'], df_sorted['GB']],
               align='left'))
])
fig_table.update_layout(title='테이블 이름 및 총 크기')
fig_table.show()